In [16]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [17]:
model=load_model('Ann_ChunPrediction.h5')

## load the encoder and scaler
with open('Encoder_Contract.pkl','rb') as file:
    Encoder_Contract=pickle.load(file)
    
with open('Encoder_Internet.pkl','rb') as file:
    Encoder_Internet=pickle.load(file)
    
with open('Label_encoder.pkl', 'rb') as file:
    label_encoder= pickle.load(file)

with open('Sclar.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [18]:
new_data = pd.DataFrame([{
    "customerID": "590-VHVEG",
    "gender": "Female",
    "SeniorCitizen": 0,
    "Partner": "Yes",
    "Dependents": "No",
    "tenure": 1,
    "PhoneService": "No",
    "MultipleLines": "No phone service",
    "InternetService": "DSL",
    "OnlineSecurity": "No",
    "OnlineBackup": "Yes",
    "DeviceProtection": "No",
    "TechSupport": "No",
    "StreamingTV": "No",
    "StreamingMovies": "No",
    "Contract": "Month-to-month",
    "PaperlessBilling": "Yes",
    "PaymentMethod": "Electronic check",
    "MonthlyCharges": 29.85,
    "TotalCharges": 29.85,
    # "Churn": "No"  # Optional if you just want to predict
}])

# Step 2: Drop columns not needed for prediction
new_data=new_data.drop(columns=['customerID','SeniorCitizen','gender','Dependents','PaperlessBilling','PaymentMethod','MonthlyCharges','PhoneService','StreamingMovies'])

In [19]:
# Transform Contract column (already returns ndarray)
Encoder_Contract_ar = Encoder_Contract.transform(new_data[['Contract']])

# Convert to DataFrame with proper column names
Encoder_Contract_DF = pd.DataFrame(
    Encoder_Contract_ar, 
    columns=Encoder_Contract.get_feature_names_out(['Contract'])
)

Encoder_Contract_DF

,Contract_Month-to-month,Contract_One year,Contract_Two year
0,1.0,0.0,0.0


In [20]:
# Transform InternetService column (returns ndarray)
Encoder_Internet_ar = Encoder_Internet.transform(new_data[['InternetService']])

# Convert to DataFrame with proper column names
Encoder_Internet_DF = pd.DataFrame(
    Encoder_Internet_ar,
    columns=Encoder_Internet.get_feature_names_out(['InternetService'])
)

Encoder_Internet_DF


,InternetService_DSL,InternetService_Fiber optic,InternetService_No
0,1.0,0.0,0.0


In [21]:
# Replace special service values with 'No'
replace_dict = {
    'MultipleLines': 'No phone service',
    'OnlineSecurity': 'No internet service',
    'OnlineBackup': 'No internet service',
    'DeviceProtection': 'No internet service',
    'TechSupport': 'No internet service',
    'StreamingTV': 'No internet service'
}

for col, val in replace_dict.items():
    if val in new_data[col].values:
        new_data[col] = new_data[col].replace(val, 'No')

In [22]:
binary_cols = ['StreamingTV', 'TechSupport', 'OnlineBackup','Partner',
               'OnlineSecurity', 'DeviceProtection', 'MultipleLines']

for col in binary_cols:
    new_data[col] = label_encoder.transform(new_data[col])

In [23]:
new_data

,Partner,tenure,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,Contract,TotalCharges
0,1,1,0,DSL,0,1,0,0,0,Month-to-month,29.85


In [24]:
new_data=pd.concat([new_data.drop(columns='InternetService',axis=1),Encoder_Internet_DF],axis=1)

In [25]:
new_data=pd.concat([new_data.drop(columns='Contract',axis=1),Encoder_Contract_DF],axis=1)

In [26]:
new_data

,Partner,tenure,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,TotalCharges,InternetService_DSL,InternetService_Fiber optic,InternetService_No,Contract_Month-to-month,Contract_One year,Contract_Two year
0,1,1,0,0,1,0,0,0,29.85,1.0,0.0,0.0,1.0,0.0,0.0


In [27]:
new_data=scaler.transform(new_data)

In [28]:
prediction=model.predict(new_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step


In [29]:
prediction

array([[0.1787922]], dtype=float32)

In [31]:
probablity=prediction[0][0]
probablity

np.float32(0.1787922)

In [32]:
if probablity > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
